In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
!pip install tensorflow-addons
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split

In [ ]:
!git clone https://github.com/naufalhisyam/TurbidityPrediction-thesis.git
os.chdir('/content/TurbidityPrediction-thesis') 

In [ ]:
images = pd.read_csv(r'./Datasets/0degree/0degInfo.csv') #load dataset info
train_df, test_df = train_test_split(images, train_size=0.9, shuffle=True, random_state=1) #Split into train and test set

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    horizontal_flip=True
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Turbidity',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

**Plotting Model Graphs**

In [ ]:
model_path = 'path'
name = 'tes'
model = keras.models.load_model(model_path) #loads saved model

pred_turbid = np.squeeze(model.predict(test_images))
true_turbid = test_images.labels
residuals = true_turbid - pred_turbid

Residual Plot

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(8,5), gridspec_kw={'width_ratios': [4, 1]})

axs[0].scatter(pred_turbid,residuals)
axs[0].set_title('Residual Plot dari Model ResNet50', fontsize=13, fontweight='bold')           
axs[0].set_ylabel('Residual (NTU)')
axs[0].set_xlabel('Predicted Turbidity (NTU)')      
axs[0].axhline(0, color='black')
axs[0].grid()

axs[1].hist(residuals, bins=40, orientation="horizontal", density=True)
axs[1].axhline(0, color='black')
axs[1].set_xlabel('Distribution')  
axs[1].yaxis.tick_right()
axs[1].grid(axis='y')

plt.subplots_adjust(wspace=0.05)
plt.savefig(f'saved_model/{name}/residualPlot_{name}.png', dpi=150)
plt.show()

Measured vs Predicted Plot

In [ ]:
fig, ax = plt.subplots()
ax.scatter(true_turbid, pred_turbid)
ax.axline([0, 0], [1, 1])
#ax.plot([true_turbid.min(), true_turbid.max()], [true_turbid.min(), true_turbid.max()], 'k--', lw=4)
ax.set_xlabel('Measured Turbidity (NTU)')
ax.set_ylabel('Predicted Turbidity (NTU)')
ax.set_title('Nilai Turbiditas Prediksi vs Observasi')
ax.set_xlim([0, 200])
ax.set_ylim([0, 200])
plt.savefig(f'plots/{name}/residualPlot_{name}.png', dpi=150)
plt.show()

Training History

In [ ]:
history = pd.read_csv(f'{model_path}/history.csv')

loss = history['loss']
val_loss = history['val_loss']
ma_error = history['mae']
val_ma_error = history['val_mae']
r2 = history['R2']
val_r2 = history['val_R2']
ms_error = history['mse']
val_ms_error = history['val_mse']

In [ ]:
epochs = range(1, len(loss) + 1)

f, axs = plt.subplots(2, 2, figsize=(12,12))
axs[0][0].plot(epochs, loss, 'tab:orange', label='train_loss (huber)')
axs[0][0].plot(epochs, val_loss, 'tab:blue', label='val_loss (huber)')
axs[0][0].set_title('Loss selama Training', fontsize=13, fontweight='bold')
axs[0][0].set_xlabel('Epoch')
axs[0][0].set_ylabel('Loss')
axs[0][0].legend(facecolor='white')
axs[0][0].grid()

axs[0][1].plot(epochs, ms_error, 'tab:orange', label='train_mse')
axs[0][1].plot(epochs, val_ms_error, 'tab:blue', label='val_mse')
axs[0][1].set_title('MSE selama Training', fontsize=13, fontweight='bold')
axs[0][1].set_xlabel('Epoch')
axs[0][1].set_ylabel('MSE')
axs[0][1].legend(facecolor='white')
axs[0][1].grid()

axs[1][0].plot(epochs, ma_error, 'tab:orange', label='train_mae')
axs[1][0].plot(epochs, val_ma_error, 'tab:blue', label='val_mae')
axs[1][0].set_title('MAE selama Training', fontsize=13, fontweight='bold')
axs[1][0].set_xlabel('Epoch')
axs[1][0].set_ylabel('MAE')
axs[1][0].legend(facecolor='white')
axs[1][0].grid()

axs[1][1].plot(epochs, r2, 'tab:orange', label='train_R2')
axs[1][1].plot(epochs, val_r2, 'tab:blue', label='val_R2')
axs[1][1].set_title('$R^2$ selama Training', fontsize=13, fontweight='bold')
axs[1][1].set_xlabel('Epoch')
axs[1][1].set_ylabel('$R^2$')
axs[1][1].legend(facecolor='white')
axs[1][1].grid()

plt.tight_layout()
plt.savefig(f'saved_model/{name}/trainPlot_{name}.png', dpi=150)
plt.show()